In [0]:
import tensorflow as tf
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU available.")
    device = torch.device("cpu")

In [0]:
!pip install transformers
#AKA huggingface library

     |████████████████████████████████| 573kB 11.6MB/s 
     |████████████████████████████████| 3.7MB 33.8MB/s 
     |████████████████████████████████| 1.0MB 47.8MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=478684225f80ce6980d59663d01ae9342c9110b9b17ba3e761aaff50779f2269
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import string
import numpy as np
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

TRAIN_FILE =  'drive/My Drive/DATA_MINING/DATA/train.csv'
TEST_FILE = 'drive/My Drive/DATA_MINING/DATA/test.csv'

train_data = pd.read_csv(TRAIN_FILE)        
test_data = pd.read_csv(TEST_FILE)

In [0]:
train_data.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [0]:
train_data.shape

(320552, 8)

In [0]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

Loading BERT tokenizer...


# **If we use Bert english version else skip to the chinese part.**



In [0]:
def clean_data(data,col):
    clean_data = []
    for i in tqdm(range(data.shape[0])):
        clean_data.append(data.iloc[i, col].translate(str.maketrans("","",string.punctuation)).lower())
    data[str('CleanContent'+str(col-4))] = clean_data

clean_data(train_data,5)
clean_data(train_data,6)

In [0]:
CLS_token = '[CLS]'
SEP_token = '[SEP]'

def make_input(data):
  temp = []
  for i in tqdm(range(data.shape[0])):
    temp.append(str(CLS_token +data['CleanContent1'][i]+SEP_token+data['CleanContent2'][i]+SEP_token))
  data['INPUTS'] = temp
  
make_input(train_data)

In [0]:
tempo = []
for j in range(train_data.shape[0]):
  tempo.append(train_data["INPUTS"][j].split())

train_data["SPLITTED"] = tempo

In [0]:
max_len = max([len(k) for k in train_data['SPLITTED'].values])
min_len = min([len(k) for k in train_data['SPLITTED'].values])
mean_len = np.mean([len(k) for k in train_data['SPLITTED'].values])
perc_50 = np.percentile([len(k) for k in train_data['SPLITTED'].values],50)
print("max = {}, min = {}, mean = {}, 50% = {}".format(max_len, min_len, mean_len, perc_50))

#  **For chinese version : Preprocessing**


In [0]:
print(train_data['title2_zh'][48417])
train_data.fillna('undefined',inplace = True)
print(train_data['title2_zh'][48417])


nan
undefined


In [0]:
labels = ['unrelated', 'agreed', 'disagreed']

temp=[]
def num_labels(data):
  for l in data['label']:
    temp.append(labels.index(l))
  data['NUM_LABEL'] = temp
num_labels(train_data)

In [0]:
max_len = []
sentences1 = train_data.title1_zh.values
sentences2 = train_data.title2_zh.values

for i in range(len(sentences1)):

    input_ids1 = tokenizer.encode(sentences1[i], add_special_tokens=True)
    input_ids2 = tokenizer.encode(sentences2[i], add_special_tokens=True)
    max_len.append(len(input_ids1)+ len((input_ids2)))

print('Mean sentence length: ', np.mean(max_len))

Mean sentence length:  52.1997429434226


In [0]:
input_ids = []
attention_masks = []
token_type_ids = []

for k in tqdm(range(train_data.shape[0])):

    encoded_dict = tokenizer.encode_plus(
                        train_data["title1_zh"][k],  
                        train_data["title2_zh"][k],                    
                        add_special_tokens = True, 
                        max_length = 64,  #We want it lowest as possible but still high enough to be relevant.         
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',  #pytorch tensors  
                   )
    
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_data.NUM_LABEL.values)
token_type_ids = torch.cat(token_type_ids, dim=0)


100%|██████████| 320552/320552 [02:57<00:00, 1810.55it/s]


In [0]:
print('Original: ', '[CLS]'+train_data["title1_zh"][0] , "[SEP]", train_data["title2_zh"][0])
print('Token IDs:', input_ids[0])
print (attention_masks[0])
print(token_type_ids[0])


Original:   2017养老保险又新增两项，农村老人人人可申领，你领到了吗 [SEP] 警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京
Token IDs: tensor([ 101, 8109, 1075, 5439,  924, 7372, 1348, 3173, 1872,  697, 7555, 8024,
        1093, 3333, 5439,  782,  782,  782, 1377, 4509, 7566, 8024,  872, 7566,
        1168,  749, 1408,  102, 6356, 3175, 6792, 6469,  100, 7881, 2338, 1920,
         833, 3680,  782, 7566,  126,  674,  100,  793, 3300, 5439,  782, 1780,
        2898, 6822,  776,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [0]:
!pip install pytorch-lamb

     |████████████████████████████████| 204kB 23.6MB/s 


In [0]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels, token_type_ids)

#Split training data into 80/20 for validation purpose
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 128
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

In [0]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-chinese", #"bert-base-uncased" for the english version
    num_labels = 3, # The number of output labels--3 for our classification. 
    output_attentions = False, 
    output_hidden_states = False, 
)

#Run the model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
# We won't use that section because it takes to much time with our setup but it would be interesting to be able to test various parameters here
from sklearn import metrics
from tensorboard.plugins.hparams import api as hp

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3, 3e-5, 1e-5]))

In [0]:
from pytorch_lamb import Lamb 

optimizer = Lamb(model.parameters(),
                  lr = 1e-3,
                  eps = 1e-8,
                  adam = False)

# optimizer = AdamW(model.parameters(),
#                   lr = 5e-5, #learning-rate
#                   eps = 1e-8)


In [0]:
from transformers import get_linear_schedule_with_warmup

epochs = 2

num_steps = int(len(train_dataloader) * epochs)
print("Steps :", num_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(num_steps/100),
                                            num_training_steps = num_steps-(int(num_steps/100)))

4008


In [0]:
# Function to calculate the accuracy of our predictions vs labels
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import tensorflow.keras.backend as K


def flat_accuracy(labels, preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [0]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    writer = SummaryWriter('runs/bert_chinese_experiment_1_epoch'+str(epoch_i))

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 1000 batches.
        if step % 200 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_token_type_ids = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation. Loss is cross-entropy based
        loss, logits = model(b_input_ids, 
                             token_type_ids=b_token_type_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_precision = 0
    total_eval_recall = 0
    #total_eval_f1 = 0
    total_eval_loss = 0
    iterator = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        #   [3]: token type
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_token_type_ids = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=b_token_type_ids, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        acc = flat_accuracy(label_ids, logits)
        #f1 = get_f1(label_ids, logits)
        y_pred = np.argmax(logits, axis=1).flatten()
        prec = precision_score(label_ids, y_pred, average='micro')
        recall = recall_score(label_ids, y_pred, average='micro')

        total_eval_accuracy += acc
        #total_eval_f1 += f1
        total_eval_precision += prec
        total_eval_recall += recall

        writer.add_scalar('Accuracy', acc, iterator)
        #writer.add_scalar('F1', f1, iterator)
        writer.add_scalar('Precision', prec, iterator)
        writer.add_scalar('Recall', recall, iterator)

        iterator+=1


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    #avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    avg_val_precision = total_eval_precision / len(validation_dataloader)
    avg_val_recall = total_eval_recall / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    #print("  F1 score: {0:.2f}".format(avg_val_f1))
    print("  Precision: {0:.2f}".format(avg_val_precision))
    print("  Recall: {0:.2f}".format(avg_val_recall))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    # training_stats.append(
    #     {
    #         'epoch': epoch_i + 1,
    #         'Training Loss': avg_train_loss,
    #         'Valid. Loss': avg_val_loss,
    #         'Valid. Accur.': avg_val_accuracy,
    #         'Valid. F1.': avg_val_f1,
    #         'Valid. Precision.': avg_val_precision,
    #         'Valid. Recall.': avg_val_recall,
    #         'Training Time': training_time,
    #         'Validation Time': validation_time
    #     }
    #)

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch   200  of  2,004.    Elapsed: 0:04:41.
  Batch   400  of  2,004.    Elapsed: 0:09:36.
  Batch   600  of  2,004.    Elapsed: 0:14:32.
  Batch   800  of  2,004.    Elapsed: 0:19:27.
  Batch 1,000  of  2,004.    Elapsed: 0:24:22.


# **Testing the model**

In [0]:
 test_data.shape

In [0]:
test_data.fillna('undefined', inplace=True)
# Create sentence lists
title1 = test_data.title1_zh.values
title2 = test_data.title2_zh.values

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
token_type_ids = []

# For every pair of sentence...
for k in tqdm(range(test_data.shape[0])):
    encoded_dict = tokenizer.encode_plus(
                        title1[k], 
                        title2[k],                     
                        add_special_tokens = True, 
                        max_length = 64,          
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, token_type_ids)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
token_type_ids = []

# For every pair of sentence...
for k in tqdm(range(test_data.shape[0])):
    encoded_dict = tokenizer.encode_plus(
                        title1[k], 
                        title2[k],                     
                        add_special_tokens = True, 
                        max_length = 64,          
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, token_type_ids)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
pred=[]

for i in range(len(predictions)):
  pred.append(np.argmax(predictions[i], axis=1).flatten())
  
tabl_predictions = []
categories = ["unrelated","agreed","disagreed"]
for i in range(len(pred)):
  for p in pred[i]:
    tabl_predictions.append(categories[p]) 

In [0]:
submission = pd.DataFrame()
submission['Id'] = test_data.id.values
submission['Category'] = tabl_predictions
submission.to_csv('sample_submission_chinese.csv',sep=',',index=False)
submission.head(50)